# Process the Data Center Load Time Series by Balancing Authority

This notebook modifies the TELL output for a given scenario to include the projected data center demands. This set of state-level data center growth projections for the year 2030 comes from Table A1 in the 2024 EPRI report "Powering Intelligence Analyzing Artificial Intelligence and Data Center Energy Consumption".

| EPRI Scenario | Annual Data Center Growth Rate |
| :-: | :-: | 
| 'Low-Growth Scenario' | 3.71% |
| 'Moderate-Growth Scenario' | 5.00% |
| 'High-Growth Scenario' | 10.00% |
| 'Higher-Growth Scenario' | 15.00% |

In [1]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell
import glob

import pandas as pd
import numpy as np


## Set the Directory Structure

In [2]:
# Identify the data input and output directories:
dc_load_input_dir =  '/Users/burl878/Documents/Code/code_repos/data_center_loads/data/'
tell_output_dir =  '/Users/burl878/Documents/IMMM/Data_Centers/Data/TELL_Output/'


## Format the EPRI Data Center Load Projections


In [6]:
# Define a function to extract the projections for a given scenario:
def extract_EPRI_load_projections(dc_load_input_dir: str, scenario: str, year_to_process: int):
    # Set the growth rate based on the EPRI scenario:
    if scenario == 'Low-Growth Scenario':
       growth_rate = 0.0371
    elif scenario == 'Moderate-Growth Scenario':
       growth_rate = 0.0500
    elif scenario == 'High-Growth Scenario':
       growth_rate = 0.1000
    elif scenario == 'Higher-Growth Scenario':
       growth_rate = 0.1500
    
    # Read in the data from the Excel file:
    epri_df = pd.read_excel((dc_load_input_dir + 'EPRI_2024_Projections.xlsx'), sheet_name='Sheet1')

    # Only keep the columns that are needed:
    epri_df = epri_df[['State', '2023 Load (MWh/y)']].copy()

    # Make a list of all of the states:
    states = epri_df['State'].unique()
        
    # Loop over the states and process their data center growth:
    for i in range(len(states)):

        # Initiate a counter and empty dataframe to store the results:
        counter = 1;
        output_df = pd.DataFrame()
    
        # Subset to just the data for the BA being processed:
        state_df = epri_df[epri_df['State'].isin([states[i]])].copy().reset_index()

        # Put the 2023 data center loads into the output dataframe:
        output_df.loc[counter, 'State'] = states[i]
        output_df.loc[counter, 'Year'] = int(2023)
        output_df.loc[counter, 'DC_Load_MWh_y'] = state_df['2023 Load (MWh/y)'][0]

        # Loop over the years 2024 to 2055 and grow the data center loads according to the growth rate:
        for year in range(2024,2060):
            # Iterate the counter by one:
            counter = counter + 1

            # Add the data for the year being processed:
            output_df.loc[counter, 'State'] = states[i]
            output_df.loc[counter, 'Year'] = int(year)
            output_df.loc[counter, 'DC_Load_MWh_y'] = (output_df.loc[counter-1, 'DC_Load_MWh_y'] + (output_df.loc[counter-1, 'DC_Load_MWh_y'] * growth_rate))

        # Add the data for 2022:
        counter = counter + 1
        output_df.loc[counter, 'State'] = states[i]
        output_df.loc[counter, 'Year'] = 2022
        output_df.loc[counter, 'DC_Load_MWh_y'] = (output_df.loc[1, 'DC_Load_MWh_y']/(1+growth_rate))

        # Aggregate the output into a new dataframe:
        if i == 0:
            aggregate_output_df = output_df
        else:
            aggregate_output_df = pd.concat([aggregate_output_df, output_df])

    # Sort the data alphabetically by state and chronologically by year:
    aggregate_output_df = aggregate_output_df.sort_values(by=['State', 'Year'])

    # Compute the mean data center hourly load assuming a flat load profile and 8760-hrs in a year:
    aggregate_output_df['Mean_Load_MWh'] = (aggregate_output_df['DC_Load_MWh_y']/8760).round(2)

    # Subset to just the data for the year being processed:
    output_df = aggregate_output_df.loc[(aggregate_output_df['Year'] == year_to_process)].reset_index(drop=True)

    # Rename some columns for consistency with TELL:
    output_df.rename(columns={'State': 'State_Name'}, inplace=True)
 
    return output_df


In [7]:
# Test the function for the scenario defined above:
epri_df = extract_EPRI_load_projections(dc_load_input_dir = dc_load_input_dir, 
                                        scenario = 'Low-Growth Scenario',
                                        year_to_process = 2030)

epri_df


,State_Name,Year,DC_Load_MWh_y,Mean_Load_MWh
0,Alabama,2030.0,1.921753e+06,219.38
1,Arizona,2030.0,8.069590e+06,921.19
2,California,2030.0,1.204208e+07,1374.67
3,Colorado,2030.0,1.948130e+06,222.39
4,Connecticut,2030.0,3.391328e+05,38.71
5,Florida,2030.0,1.786099e+06,203.89
6,Georgia,2030.0,7.969093e+06,909.71
7,Hawaii,2030.0,1.130443e+04,1.29
8,Idaho,2030.0,1.921753e+05,21.94
9,Illinois,2030.0,9.614152e+06,1097.51


In [8]:
# Calculate the sum of data center loads in 2030:
print(('Total data center loads from EPRI = ' + str(((epri_df['DC_Load_MWh_y'].sum()) / 1000000).round(2)) + ' TWh'))


Total data center loads from EPRI = 196.31 TWh


## Leverage the TELL 'execute_forward' Function as the Basis for Projection

The `tell.execute_forward` function takes the .csv files produced by the TELL MLP model and distributes the predicted load to the counties that each balancing authority (BA) operates in. The county-level hourly loads are then summed to the state-level and scaled to match the state-level annual loads produced by GCAM-USA. The function is modified below to merge in the data center load projections. The function is defined and tested here and then ran iteratively in the next section.


In [9]:
def execute_forward(year_to_process: str, gcam_target_year: str, scenario_to_process: str, epri_scenario: str, data_output_dir: str,
                    gcam_usa_input_dir: str, map_input_dir: str, mlp_input_dir: str, pop_input_dir: str):

    # Set the growth rate based on the EPRI scenario:
    if epri_scenario == 'Low-Growth Scenario':
       epri_scenario_short = 'low_growth'
    elif epri_scenario == 'Moderate-Growth Scenario':
       epri_scenario_short = 'moderate_growth'
    elif epri_scenario == 'High-Growth Scenario':
       epri_scenario_short = 'high_growth'
    elif epri_scenario == 'Higher-Growth Scenario':
       epri_scenario_short = 'higher_growth'
    
    # Set the data output directory:
    data_output_dir_full = os.path.join(data_output_dir, scenario_to_process, gcam_target_year)

    # Check if the data output directory exists and if not then create it:
    if os.path.exists(data_output_dir_full) is False:
        os.makedirs(data_output_dir_full)
        
    # Load in the sample GCAM-USA output file and subset the data to only the "year_to_process":
    gcam_usa_df = tell.extract_gcam_usa_loads(scenario_to_process = scenario_to_process,
                                         gcam_usa_input_dir = gcam_usa_input_dir)
    gcam_usa_df = gcam_usa_df[gcam_usa_df['Year'] == int(gcam_target_year)]

    # Load in the most recent (i.e., 2019) BA service territory mapping file:
    ba_mapping_df = pd.read_csv((os.path.join(map_input_dir, 'ba_service_territory_2019.csv')), index_col=None, header=0)

    # Read in the population projection dataset for the scenario being processed:
    population_df = tell.process_population_scenario(scenario_to_process, pop_input_dir)

    # Subset to only the year being processed:
    if (scenario_to_process == 'historic') and (int(year_to_process) < 2000):
       population_df = population_df.loc[(population_df['Year'] == 2000)]
    else:
       population_df = population_df.loc[(population_df['Year'] == int(year_to_process))]

    # Only keep the columns that are needed:
    population_df = population_df[['County_FIPS', 'Population']].copy()

    # Merge the ba_mapping_df and population_df dataframes. Compute the fraction of the total population in each BA that lives in a given county:
    mapping_df = ba_mapping_df.merge(population_df, on=['County_FIPS'])
    mapping_df = mapping_df.sort_values('BA_Number')
    mapping_df['BA_Population_Sum'] = mapping_df.groupby('BA_Code')['Population'].transform('sum')
    mapping_df['BA_Population_Fraction'] = mapping_df['Population'] / mapping_df['BA_Population_Sum']
    mapping_df = mapping_df.dropna()

    # Create a list of all of the MLP output files in the "mlp_input_dir" and aggregate the files in that list:
    mlp_filelist = sorted(glob.glob(os.path.join(mlp_input_dir, scenario_to_process, year_to_process, '*_mlp_output.csv')))
    mlp_output_df = tell.aggregate_mlp_output_files(mlp_filelist)

    # Merge the "mapping_df" with "mlp_output_df" using BA codes to merge on:
    joint_mlp_df = pd.merge(mlp_output_df, mapping_df, on='BA_Code')

    # Scale the BA loads in each county by the fraction of the BA's total population that lives there:
    joint_mlp_df['County_BA_Load_MWh'] = joint_mlp_df['Total_BA_Load_MWh'].mul(joint_mlp_df['BA_Population_Fraction'])

    # Sum the county-level hourly loads into annual state-level total loads and convert that value from MWh to TWh:
    joint_mlp_df['TELL_State_Annual_Load_TWh'] = (joint_mlp_df.groupby('State_FIPS')['County_BA_Load_MWh'].transform('sum')) / 1000000

    # Add a column with the state-level annual total loads from GCAM-USA:
    joint_mlp_df = pd.merge(joint_mlp_df, gcam_usa_df[['State_FIPS', 'GCAM_USA_State_Annual_Load_TWh']], on='State_FIPS', how='left')

    # Compute the state-level scaling factors that force TELL annual loads to match GCAM-USA annual loads:
    joint_mlp_df['State_Scaling_Factor'] = joint_mlp_df['GCAM_USA_State_Annual_Load_TWh'].div(joint_mlp_df['TELL_State_Annual_Load_TWh'])

    # Apply those scaling factors to the "County_BA_Load_MWh" variable:
    joint_mlp_df['County_BA_Load_MWh_Scaled'] = joint_mlp_df['County_BA_Load_MWh'].mul(joint_mlp_df['State_Scaling_Factor'])

    # Extract the EPRI data center loads using the function defined above:
    epri_df = extract_EPRI_load_projections(dc_load_input_dir = dc_load_input_dir, 
                                            scenario = epri_scenario,
                                            year_to_process = int(year_to_process))

    # Merge the TELL and data center dataframes together based on common state names:
    joint_mlp_df = joint_mlp_df.merge(epri_df, on=['State_Name'], how='left')

    # Replacing missing or negative loads with NaN:
    joint_mlp_df.loc[~(joint_mlp_df['Mean_Load_MWh'] > 0), 'Mean_Load_MWh'] = 0

    # Calculate the total population in each state/time combination:
    joint_mlp_df['State_Population_Sum'] = joint_mlp_df.groupby(['Time_UTC', 'State_Name'])['Population'].transform('sum')
    
    # Calculate the fraction of people that live in each county as a function of state population:
    joint_mlp_df['County_Population_Fraction'] = joint_mlp_df['Population'].div(joint_mlp_df['State_Population_Sum'])
    
    # Calculate the fraction of the data center load in each state that should be attributed to each BA/state combination:
    joint_mlp_df['County_DC_Load_MWh'] = joint_mlp_df['County_Population_Fraction'] * joint_mlp_df['Mean_Load_MWh']

    # Calculate the new hourly loads by summing the scaled TELL loads with data centers:
    joint_mlp_df['Total_Load_MWh'] = joint_mlp_df['County_DC_Load_MWh'] + joint_mlp_df['County_BA_Load_MWh_Scaled']

    # Make a list of all of the BAs in "ba_output_df":
    bas = joint_mlp_df['BA_Code'].unique()

    # Loop over the BAs and process their data into an output file:
    for i in range(len(bas)):

        # Subset to just the data for the BA being processed:
        output_df = joint_mlp_df[joint_mlp_df['BA_Code'].isin([bas[i]])].copy()

        # Sum the county loads as a function of time:
        output_df['Raw_TELL_BA_Load_MWh'] = output_df.groupby('Time_UTC')['County_BA_Load_MWh'].transform('sum')
        output_df['Scaled_TELL_BA_Load_MWh'] = output_df.groupby('Time_UTC')['County_BA_Load_MWh_Scaled'].transform('sum')
        output_df['Scaled_TELL_BA_Load_with_DC_MWh'] = output_df.groupby('Time_UTC')['Total_Load_MWh'].transform('sum')

        # Subset and reorder the columns, drop duplicates, fill in missing values, and sort chronologically:
        output_df = output_df[['BA_Code', 'BA_Number', 'Time_UTC', 'Raw_TELL_BA_Load_MWh', 'Scaled_TELL_BA_Load_MWh', 'Scaled_TELL_BA_Load_with_DC_MWh']]
        output_df = output_df.drop_duplicates()
        output_df = output_df.fillna(-9999)
        output_df = output_df.sort_values('Time_UTC')

        # Round off the values to make the output file more readable:
        output_df['BA_Number'] = output_df['BA_Number'].round(0)
        output_df['Raw_TELL_BA_Load_MWh'] = output_df['Raw_TELL_BA_Load_MWh'].round(2)
        output_df['Scaled_TELL_BA_Load_MWh'] = output_df['Scaled_TELL_BA_Load_MWh'].round(2)
        output_df['Scaled_TELL_BA_Load_with_DC_MWh'] = output_df['Scaled_TELL_BA_Load_with_DC_MWh'].round(2)

        # Aggregate the output into a new dataframe:
        if i == 0:
           aggregate_output_df = output_df
        else:
           aggregate_output_df = pd.concat([aggregate_output_df, output_df])
    
    # Sort the data alphabetically by BA:
    aggregate_output_df = aggregate_output_df.sort_values(by=["BA_Code", "Time_UTC"])

    # Generate the .csv output file name:
    csv_output_filename = os.path.join(data_output_dir_full, 'TELL_BA_Loads_' + epri_scenario_short + '.csv')
 
    # Write out the dataframe to a .csv file:
    aggregate_output_df.to_csv(csv_output_filename, sep=',', index=False)

    # Sum the county-level hourly loads with data centers into annual state-level total loads and convert that value from MWh to TWh:
    joint_mlp_df['DC_State_Annual_Load_TWh'] = (joint_mlp_df.groupby('State_FIPS')['Total_Load_MWh'].transform('sum')) / 1000000

    # Make a copy of the necessary variables, drop the duplicates, and add in the "year_to_process":
    state_output_df = joint_mlp_df[['State_FIPS', 'State_Name', 'TELL_State_Annual_Load_TWh', 'GCAM_USA_State_Annual_Load_TWh','State_Scaling_Factor','DC_State_Annual_Load_TWh']].copy(deep=False)
    state_output_df = state_output_df.drop_duplicates()
    state_output_df['Year'] = year_to_process

    # Calculate the scaled TELL loads:
    state_output_df['Scaled_TELL_Load_TWh'] = state_output_df['TELL_State_Annual_Load_TWh'].mul(state_output_df['State_Scaling_Factor'])
    
    # Calculate the annual state-level loads due to data centers:
    state_output_df['DC_Load_MWh'] = (state_output_df['DC_State_Annual_Load_TWh'] - state_output_df['Scaled_TELL_Load_TWh']) * 1000000

    # Rename the columns to make them more readable:
    state_output_df.rename(columns={'TELL_State_Annual_Load_TWh': 'Raw_TELL_Load_TWh',
                                    'GCAM_USA_State_Annual_Load_TWh': 'GCAM_USA_Load_TWh',
                                    'State_Scaling_Factor': 'Scaling_Factor',
                                    'DC_State_Annual_Load_TWh': 'Loads_With_DC_TWh',
                                    'DC_Load_MWh': 'Total_DC_Load_MWh'}, inplace=True)

    # Round off the values to make the output file more readable:
    state_output_df['State_FIPS'] = state_output_df['State_FIPS'].round(0)
    state_output_df['Raw_TELL_Load_TWh'] = state_output_df['Raw_TELL_Load_TWh'].round(5)
    state_output_df['GCAM_USA_Load_TWh'] = state_output_df['GCAM_USA_Load_TWh'].round(5)
    state_output_df['Scaled_TELL_Load_TWh'] = state_output_df['Scaled_TELL_Load_TWh'].round(5)
    state_output_df['Loads_With_DC_TWh'] = state_output_df['Loads_With_DC_TWh'].round(5)
    state_output_df['Total_DC_Load_MWh'] = state_output_df['Total_DC_Load_MWh'].round(0)
    state_output_df['Scaling_Factor'] = state_output_df['Scaling_Factor'].round(5)

    # Reorder the columns, fill in missing values, and sort alphabetically by state name:
    state_output_df = state_output_df[['Year', 'State_Name', 'State_FIPS', 'Scaling_Factor', 'GCAM_USA_Load_TWh', 'Raw_TELL_Load_TWh', 'Scaled_TELL_Load_TWh', 'Loads_With_DC_TWh', 'Total_DC_Load_MWh']]
    state_output_df = state_output_df.fillna(-9999)
    state_output_df = state_output_df.sort_values('State_Name')

    # Generate the .csv output file name:
    csv_output_filename = os.path.join(data_output_dir_full, 'TELL_State_Summary_Data_' + epri_scenario_short + '.csv')
 
    # Write out the dataframe to a .csv file:
    state_output_df.to_csv(csv_output_filename, sep=',', index=False)
    
    return aggregate_output_df


In [10]:
# Test the function to run the TELL model forward in time:
output_df = execute_forward(year_to_process = '2030',
                            gcam_target_year = '2030', 
                            scenario_to_process = 'rcp85hotter_ssp3',
                            epri_scenario = 'Low-Growth Scenario',
                            data_output_dir = tell_output_dir,
                            gcam_usa_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/gcamusa_data',
                            map_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/ba_service_territory_data',
                            mlp_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/outputs/mlp_output',
                            pop_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/population_data')

output_df


,BA_Code,BA_Number,Time_UTC,Raw_TELL_BA_Load_MWh,Scaled_TELL_BA_Load_MWh,Scaled_TELL_BA_Load_with_DC_MWh
1419120,AEC,189.0,2030-01-01 00:00:00,449.55,496.69,559.59
1419170,AEC,189.0,2030-01-01 01:00:00,465.18,513.95,576.86
1419220,AEC,189.0,2030-01-01 02:00:00,468.38,517.49,580.40
1419270,AEC,189.0,2030-01-01 03:00:00,441.15,487.40,550.31
1419320,AEC,189.0,2030-01-01 04:00:00,415.94,459.55,522.46
...,...,...,...,...,...,...
38859190,WAUW,19610.0,2030-12-31 19:00:00,99.15,100.87,131.23
38859224,WAUW,19610.0,2030-12-31 20:00:00,98.35,100.06,130.42
38859258,WAUW,19610.0,2030-12-31 21:00:00,97.70,99.40,129.76
38859292,WAUW,19610.0,2030-12-31 22:00:00,97.71,99.41,129.77


In [11]:
# Calculate the difference in loads due to data centers:
output_df['DC_Load_MWh'] = output_df['Scaled_TELL_BA_Load_with_DC_MWh'] - output_df['Scaled_TELL_BA_Load_MWh']

# Calculate the sum of data center loads in 2030:
print(('Total data center loads = ' + str(((output_df['DC_Load_MWh'].sum()) / 1000000).round(2)) + ' TWh'))


Total data center loads = 196.29 TWh


## Loop Over All Scenarios and Years


In [12]:
# Loop over all the EPRI scenarios:
for epri_scenario in ['Low-Growth Scenario', 'Moderate-Growth Scenario', 'High-Growth Scenario', 'Higher-Growth Scenario']:
    # Loop over the IM3 scenarios:
    for im3_scenario in ['rcp45hotter_ssp3', 'rcp45hotter_ssp5', 'rcp85hotter_ssp3', 'rcp85hotter_ssp5']:
        # Loop over the years:
        for year in range(2022,2037):
            execute_forward(year_to_process = str(year),
                            gcam_target_year = str(year), 
                            scenario_to_process = im3_scenario,
                            epri_scenario = epri_scenario,
                            data_output_dir = tell_output_dir,
                            gcam_usa_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/gcamusa_data',
                            map_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/ba_service_territory_data',
                            mlp_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/outputs/mlp_output',
                            pop_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/population_data')
